In [0]:
!pip install --upgrade keras
!pip install --upgrade tensorflow

In [2]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

Using TensorFlow backend.


['/job:localhost/replica:0/task:0/device:GPU:0']

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

BASE_DIR = '/content/gdrive/My Drive/Colab Notebooks/Fashion-MNIST/'

train_file = BASE_DIR + "fashion-mnist_train.csv"
test_file  = BASE_DIR + "fashion-mnist_test.csv"

train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

x_train = train_data.values[:,1:]
y_train = train_data.label

x_test = test_data.values[:,1:]
y_test = test_data.label

In [0]:
from keras.utils import to_categorical

img_rows, img_cols = 28, 28
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [0]:
from keras import layers
from keras.models import Model
from keras.losses import categorical_crossentropy

inputs = layers.Input(shape=input_shape)

# First block of AlexNet
x = layers.Conv2D(96, (11, 11), activation='relu', strides=(4, 4), padding='same', name="b1-c1")(inputs)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name="b1-mp1")(x)
x = layers.BatchNormalization(name="b1-bn1")(x)

# First block of VggNet
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='b2-c1')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same', name='b2-c2')(x)
x = layers.MaxPooling2D((2, 2), strides=(2, 2), name='b2-mp1')(x)

# Dephwise convolution block of MobileNet
x = layers.ZeroPadding2D(padding=((0,1),(0, 1)), name='b3-zp1')(x)
x = layers.DepthwiseConv2D((3, 3), strides=(2, 2), padding='same', name='b3-dc1')(x)
x = layers.BatchNormalization(name='b3-bn1')(x)
x = layers.ReLU(6., name='b3-r1')(x)
x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', name='b3-c1')(x)
x = layers.BatchNormalization(name='b3-bn2')(x)
x = layers.ReLU(6., name='b3-r2')(x)

last_layer = x
# Residual_Block of ResNet
x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal', name='b4-c1')(x)
x = layers.BatchNormalization(name='b4-bn1')(x)
x = layers.Activation('relu',name='b4-r1')(x)
x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal', name='b4-c2')(x)
x = layers.BatchNormalization(name='b4-bn2')(x)
x = layers.Activation('relu', name='b4-r2')(x)
x = layers.Conv2D(64, (3, 3), strides=(2, 2), padding='same', kernel_initializer='he_normal', name='b4-c3')(x)
x = layers.BatchNormalization(name='b4-bn3')(x)
x = layers.add([x, last_layer], name='b4-a1')
x = layers.Activation('relu', name='b4-r3')(x)

# Transition block of DensNet
x = layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='b5-bn1')(x)
x = layers.Activation('relu', name='b5-r1')(x)
x = layers.Conv2D(int(K.int_shape(x)[3] * 0.5),1 , use_bias=False, name='b5-c1')(x)

# Classification block of GoogLeNet
x = layers.GlobalAveragePooling2D(name='b6-gap1')(x)
predictions = layers.Dense(10, activation='softmax', name='b6-d1')(x)
  
model = Model(inputs=inputs, outputs=predictions)

model.compile(loss=categorical_crossentropy,
              optimizer='sgd',
              metrics=['accuracy'])

In [10]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 28, 28, 1)    0                                            
__________________________________________________________________________________________________
b1-c1 (Conv2D)                  (None, 7, 7, 96)     11712       input_1[0][0]                    
__________________________________________________________________________________________________
b1-mp1 (MaxPooling2D)           (None, 4, 4, 96)     0           b1-c1[0][0]                      
__________________________________________________________________________________________________
b1-bn1 (BatchNormalization)     (None, 4, 4, 96)     384         b1-mp1[0][0]                     
__________________________________________________________________________________________________
b2-c1 (Con

In [8]:
model.fit(x_train, y_train,
          batch_size=64,
          epochs=10,
          verbose=1,
          validation_data=(x_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 18s 308us/step - loss: 0.5559 - acc: 0.8093 - val_loss: 0.4249 - val_acc: 0.8428
Epoch 2/10
60000/60000 [==============================] - 17s 278us/step - loss: 0.3782 - acc: 0.8653 - val_loss: 0.3802 - val_acc: 0.8643
Epoch 3/10
60000/60000 [==============================] - 16s 270us/step - loss: 0.3340 - acc: 0.8798 - val_loss: 0.3852 - val_acc: 0.8580
Epoch 4/10
60000/60000 [==============================] - 16s 264us/step - loss: 0.3079 - acc: 0.8888 - val_loss: 0.3050 - val_acc: 0.8876
Epoch 5/10
60000/60000 [==============================] - 16s 274us/step - loss: 0.2841 - acc: 0.8957 - val_loss: 0.3003 - val_acc: 0.8892
Epoch 6/10
60000/60000 [==============================] - 16s 266us/step - loss: 0.2686 - acc: 0.9015 - val_loss: 0.3161 - val_acc: 0.8800
Epoch 7/10
60000/60000 [==============================] - 16s 263us/step - loss: 0.2546 - acc: 0.9062 - val_loss: 0.

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.3055308569073677
Test accuracy: 0.8932
